<a href="https://colab.research.google.com/github/HMendesDev/Data-Science/blob/main/LinearRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Carregando e Conhecendo os dados

In [2]:
arq = "/content/drive/MyDrive/Data Science/Regressão/house_price_regression_dataset.csv"

In [3]:
df = pd.read_csv(arq, sep=",") # Carregando os dados

In [4]:
df.head() # 5 primeiras linhas

,Square_Footage,Num_Bedrooms,Num_Bathrooms,Year_Built,Lot_Size,Garage_Size,Neighborhood_Quality,House_Price
0,1360,2,1,1981,0.599637,0,5,2.623829e+05
1,4272,3,3,2016,4.753014,1,6,9.852609e+05
2,3592,1,2,2016,3.634823,0,9,7.779774e+05
3,966,1,2,1977,2.730667,1,8,2.296989e+05
4,4926,2,1,1993,4.699073,0,8,1.041741e+06


In [5]:
df.tail() # 5 últimas linhas

,Square_Footage,Num_Bedrooms,Num_Bathrooms,Year_Built,Lot_Size,Garage_Size,Neighborhood_Quality,House_Price
995,3261,4,1,1978,2.165110,2,10,701493.997069
996,3179,1,2,1999,2.977123,1,10,683723.160704
997,2606,4,2,1962,4.055067,0,2,572024.023634
998,4723,5,2,1950,1.930921,0,7,964865.298639
999,3268,4,2,1983,3.108790,2,2,742599.253332


In [6]:
df.columns # Colunas de nosso dataset

Index(['Square_Footage', 'Num_Bedrooms', 'Num_Bathrooms', 'Year_Built',
       'Lot_Size', 'Garage_Size', 'Neighborhood_Quality', 'House_Price'],
      dtype='object')

# Limpeza e Transformação

In [7]:
df[df.duplicated()] # Verificando registros duplicados

,Square_Footage,Num_Bedrooms,Num_Bathrooms,Year_Built,Lot_Size,Garage_Size,Neighborhood_Quality,House_Price


In [10]:
df.isnull().sum().sum() # Verificando registros nulos

np.int64(0)

In [13]:
df.shape[0] * df.shape[1] # Quantidade de registros em nosso dataset

8000

Nosso dataset não tem linhas duplicadas e nem registros nulos, porém vamos explorar posteriormente se encontramos registros sujos ou seja aqueles que não apresentam uma informação válida do dado em si e temos no total 8000 mil registros

In [11]:
df.info() # Verificando informações do dataset

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Square_Footage        1000 non-null   int64  
 1   Num_Bedrooms          1000 non-null   int64  
 2   Num_Bathrooms         1000 non-null   int64  
 3   Year_Built            1000 non-null   int64  
 4   Lot_Size              1000 non-null   float64
 5   Garage_Size           1000 non-null   int64  
 6   Neighborhood_Quality  1000 non-null   int64  
 7   House_Price           1000 non-null   float64
dtypes: float64(2), int64(6)
memory usage: 62.6 KB


Nosso dataset consiste em 8 variáveis, 1 delas sendo a variável target (House_price) e 7 delas que são características que podem influenciar na nossa variável target, vamos conhecer:

Square_Footage: Tamanho da casa em metros quadrados

Num_Bedrooms: O número de quartos na casa

Num_Bathrooms: O número de banheiros na casa

Year_Built: Ano em que construiu a casa

Lot_Size: Tamanho do lote em que a casa foi construida

Garage_Size: Quantidade de carros que cabem na garagem

Neighborhood_Quality: Classificação da qualidade do bairro em uma escala de 1 a 10

House_Price: O preço da casa (target)